In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
num_nodes = 4

num_txns = 72000
fialed_node_id = 9
write_percent = 10
usecase = 'stack'
operationrate =1000000000

In [3]:
gcp_username = 'tejas'

In [4]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create tmane-'+f"{i:03}"+' \
    --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-a \
    --machine-type=e2-highcpu-4 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20241002-130909,image=projects/rhel-cloud/global/images/rhel-8-v20240910,mode=rw,size=20,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-000].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-003].


NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
tmane-000  us-central1-a  e2-highcpu-4               10.128.0.36  35.224.170.69  RUNNING


 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP  STATUS
tmane-003  us-central1-a  e2-highcpu-4               10.128.0.10  34.71.42.92  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-001].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-002].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.

 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
tmane-001  us-central1-a  e2-highcpu-4               10.128.0.49  104.154.22.74  RUNNING
NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
tmane-002  us-central1-a  e2-highcpu-4               10.128.0.63  35.184.1.224  RUNNING


In [5]:
time.sleep(60)

In [6]:
os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')


[main 18d7b11] test
 2 files changed, 1923 insertions(+), 583 deletions(-)


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
 + f95fd04...18d7b11 HEAD -> main (forced update)


0

In [7]:
def clean_node(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd; sudo rm -r *"'
    )
    output = subprocess.getoutput(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(clean_node)(i) for i in range(num_nodes))

rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory


In [8]:
def git_clone(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_clone)(i) for i in range(num_nodes))


gcloud compute ssh --zone "us-central1-a" "tmane-000" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-001" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-003" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Re

Operation aborted.


Last metadata expiration check: 0:00:21 ago on Thu 09 Oct 2025 08:17:42 PM UTC.
Dependencies resolved.
 Package                      Arch    Version                                      Repository                                   Size
Installing:
 kernel                       x86_64  4.18.0-553.78.1.el8_10                       rhui-rhel-8-for-x86_64-baseos-rhui-rpms      11 M
Upgrading:
 NetworkManager               x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     2.3 M
 NetworkManager-libnm         x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     1.9 M
 NetworkManager-team          x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     160 k
 NetworkManager-tui           x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     356 k
 audit                        x86_64  3.1.2-1.el8_10.1                      

Operation aborted.
Operation aborted.


Last metadata expiration check: 0:00:22 ago on Thu 09 Oct 2025 08:17:44 PM UTC.
Last metadata expiration check: 0:00:22 ago on Thu 09 Oct 2025 08:17:44 PM UTC.
Dependencies resolved.
 Package                      Arch    Version                                      Repository                                   Size
Installing:
 kernel                       x86_64  4.18.0-553.78.1.el8_10                       rhui-rhel-8-for-x86_64-baseos-rhui-rpms      11 M
Upgrading:
 NetworkManager               x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     2.3 M
 NetworkManager-libnm         x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     1.9 M
 NetworkManager-team          x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     160 k
 NetworkManager-tui           x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     35

...

 
  Installing       : git-core-2.43.7-1.el8_10.x86_64                      42/48 
  Installing       : perl-libs-4:5.26.3-423.el8_10.x86_64                 30/48 
  Installing       : perl-parent-1:0.237-1.el8.noarch                     31/48 
  Installing       : perl-macros-4:5.26.3-423.el8_10.x86_64               32/48 
  Installing       : perl-Unicode-Normalize-1.25-396.el8.x86_64           33/48 
  Installing       : perl-Text-Tabs+Wrap-2013.0523-395.el8.noarch         34/48 
  Installing       : perl-constant-1.33-396.el8.noarch                    35/48
  Installing       : perl-PathTools-3.74-1.el8.x86_64                     36/48 
  Installing       : perl-threads-shared-1.58-2.el8.x86_64                37/48 
  Installing       : perl-threads-1:2.21-2.el8.x86_64                     38/48 
  Installing       : perl-File-Path-2.15-2.el8.noarch                     39/48 
0 Installing       : perl-IO-1.38-423.el8_10.x86_64                       40/48
 
  Installing       : git-c

Cloning into 'Optimistic-Replication'...
Cloning into 'Optimistic-Replication'...


0
0
 
  Installing       : git-core-doc-2.43.7-1.el8_10.noarch                  43/48 
  Installing       : perl-Error-1:0.17025-2.el8.noarch                    44/48 
  Installing       : perl-TermReadKey-2.37-7.el8.x86_64                   45/48 
  Installing       : emacs-filesystem-1:26.1-15.el8_10.noarch             46/48 
  Installing       : perl-Git-2.43.7-1.el8_10.noarch                      47/48 
  Installing       : git-2.43.7-1.el8_10.x86_64                           48/48
  Running scriptlet: git-2.43.7-1.el8_10.x86_64                           48/48
  Verifying        : perl-IO-Socket-IP-0.39-5.el8.noarch                   1/48 
  Verifying        : perl-libnet-3.11-3.el8.noarch                         2/48 
  Verifying        : perl-Digest-1.17-395.el8.noarch                       3/48 
  Verifying        : perl-Error-1:0.17025-2.el8.noarch                     4/48 
  Verifying        : perl-TermReadKey-2.37-7.el8.x86_64                    5/48 
  Verifying        : per

Cloning into 'Optimistic-Replication'...


0


In [9]:
def kill_code(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "sudo killall main-replication-protocols"'
    )
    output = subprocess.getoutput(command)
    print(output)


## RESTART

In [250]:

for i in range(num_nodes):
    kill_code(i)

    
os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')

def git_pull(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_pull)(i) for i in range(num_nodes))

main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
[main c443f74] test
 2 files changed, 149 insertions(+), 99 deletions(-)


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
   5deae34..c443f74  HEAD -> main


gcloud compute ssh --zone "us-central1-a" "tmane-000" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"

gcloud compute ssh --zone "us-central1-a" "tmane-003" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-001" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"


From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   5deae34..c443f74  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   5deae34..c443f74  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   5deae34..c443f74  main       -> origin/main


Updating 5deae34..c443f74
Fast-forward
Updating 5deae34..c443f74
Fast-forward
 OptimisticReplication.ipynb        | 201 +++++++++++++++++++------------------
 tcp/main-replication-protocols.cpp |  47 ++++++++-
 2 files changed, 149 insertions(+), 99 deletions(-)
 OptimisticReplication.ipynb        | 201 +++++++++++++++++++------------------
 tcp/main-replication-protocols.cpp |  47 ++++++++-
 2 files changed, 149 insertions(+), 99 deletions(-)
0
0
Updating 5deae34..c443f74
Fast-forward
 OptimisticReplication.ipynb        | 201 +++++++++++++++++++------------------
 tcp/main-replication-protocols.cpp |  47 ++++++++-
 2 files changed, 149 insertions(+), 99 deletions(-)
Updating 5deae34..c443f74
Fast-forward
 OptimisticReplication.ipynb        | 201 +++++++++++++++++++------------------
 tcp/main-replication-protocols.cpp |  47 ++++++++-
 2 files changed, 149 insertions(+), 99 deletions(-)
0
0


From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   5deae34..c443f74  main       -> origin/main


In [251]:
def setup(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication; cp optrep_script.sh ../optrep_script.sh; cd ../; sh optrep_script.sh {num_nodes} {num_txns} {write_percent} {usecase} > /home/{gcp_username}/log.txt"'
    )
    os.system(command)

    

In [252]:
num_txns

72000

In [253]:
results = Parallel(n_jobs=60, prefer="threads")(delayed(setup)(i) for i in range(num_nodes))

In file included from stack-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
mkdir: cannot create directory ‘workload’: File exists
mkdir: cannot create directory ‘4-72000-10’: File exists
mkdir: cannot create directory ‘stack’: File exists
In file included from stack-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
In file included from stack-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
mkdir: cannot create directory ‘workload’: File exists
mkdir: cannot create directory ‘4-72000-10’: File exists
mkdir: cannot create directory ‘stack’: File exists
In file i

In [254]:
result = subprocess.check_output('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)"', shell=True, text=True)

# Convert the output to a list by splitting on newlines
ip_list = result.splitlines()

In [255]:
ip_list

['10.128.0.36', '10.128.0.49', '10.128.0.63', '10.128.0.10']

In [256]:
def get_sorted_instance_ips():
    result = subprocess.check_output(
        "gcloud compute instances list --filter='name~^tmane-' "
        "--format='value(name,networkInterfaces[0].networkIP)'",
        shell=True
    )
    lines = result.decode().splitlines()
    # Parse into list of tuples: (name, ip)
    name_ip_pairs = [line.strip().split() for line in lines]
    # Sort by name: optrep-000, optrep-001, ...
    sorted_pairs = sorted(name_ip_pairs, key=lambda x: x[0])
    # Extract sorted IPs
    sorted_ips = [ip for name, ip in sorted_pairs]
    return sorted_ips

ip_list = get_sorted_instance_ips()
ip_string = " ".join(ip_list)

In [257]:
ip_string

'10.128.0.36 10.128.0.49 10.128.0.63 10.128.0.10'

In [258]:
def run_code(i):
    output_dir = f"../../wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}"
    output_file = f"{output_dir}/result{i}.out"
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication/tcp/build; '
        f'sudo bash -c \''
        f'mkdir -p {output_dir}; '
        f'nohup ./main-replication-protocols {i+1} {num_nodes} {num_txns} {write_percent} {usecase} {operationrate} {fialed_node_id} {ip_string} > {output_file} 2>&1 &'
        f'\'"'
    )
    print(command)
    output = subprocess.getoutput(command)
    print(output)



In [259]:
# results = Parallel(n_jobs=60)(delayed(run_code)(i) for i in range(num_nodes))

for i in range(num_nodes):
    run_code(i)


gcloud compute ssh --zone us-central1-a tejas@tmane-000 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-72000-10/stack; nohup ./main-replication-protocols 1 4 72000 10 stack 1000000000 9 10.128.0.36 10.128.0.49 10.128.0.63 10.128.0.10 > ../../wellcoordination/workload/4-72000-10/stack/result0.out 2>&1 &'"

gcloud compute ssh --zone us-central1-a tejas@tmane-001 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-72000-10/stack; nohup ./main-replication-protocols 2 4 72000 10 stack 1000000000 9 10.128.0.36 10.128.0.49 10.128.0.63 10.128.0.10 > ../../wellcoordination/workload/4-72000-10/stack/result1.out 2>&1 &'"

gcloud compute ssh --zone us-central1-a tejas@tmane-002 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-7

In [260]:
time.sleep(30)  # Let it run for 30 seconds
kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
time.sleep(60) 

In [261]:
for i in range(num_nodes):
    kill_code(i)


main-replication-protocols: no process found

main-replication-protocols: no process found


In [262]:

if not os.path.exists('../optrep_outputs'):
    os.mkdir('../optrep_outputs')

for i in [0,1, 2, 3]:
    command = (
        f'gcloud compute scp --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/stack/result{i}.out '
        f'../optrep_outputs/{i}.txt '
        f'--project ucr-ursa-major-lesani-lab'
    )
    os.system(command)
    # break

In [263]:
# os.system('gcloud compute instances delete --zone=us-central1-a --quiet --project=ucr-ursa-major-lesani-lab $(gcloud compute instances list --project=ucr-ursa-major-lesani-lab --filter="name~^tmane-" --format="value(name)")')